
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"


# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[6]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
### Handling categorical features
from pyspark.ml.feature import StringIndexer

In [0]:
### Handling categorical features
indexer=StringIndexer(inputCols=["sex","smoker","day","time"], outputCols=["sex_indexed","smoker_indexed","day_indexed","time_indexed"])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_r.columns

Out[10]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','sex_indexed','size','smoker_indexed','day_indexed','time_indexed'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,1.0,2.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,0.0,3.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,0.0,3.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,0.0,2.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,1.0,4.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
finalized_data=output.select("Independent Features","total_bill")

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,1.0,2.0,0.0...|     16.99|
|[1.66,0.0,3.0,0.0...|     10.34|
|[3.5,0.0,3.0,0.0,...|     21.01|
|[3.31,0.0,2.0,0.0...|     23.68|
|[3.61,1.0,4.0,0.0...|     24.59|
|[4.71,0.0,4.0,0.0...|     25.29|
|[2.0,0.0,2.0,0.0,...|      8.77|
|[3.12,0.0,4.0,0.0...|     26.88|
|[1.96,0.0,2.0,0.0...|     15.04|
|[3.23,0.0,2.0,0.0...|     14.78|
|[1.71,0.0,2.0,0.0...|     10.27|
|[5.0,1.0,4.0,0.0,...|     35.26|
|[1.57,0.0,2.0,0.0...|     15.42|
|[3.0,0.0,4.0,0.0,...|     18.43|
|[3.02,1.0,2.0,0.0...|     14.83|
|[3.92,0.0,2.0,0.0...|     21.58|
|[1.67,1.0,3.0,0.0...|     10.33|
|[3.71,0.0,3.0,0.0...|     16.29|
|[3.5,1.0,3.0,0.0,...|     16.97|
|(6,[0,2],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
## Train Test Split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[18]: DenseVector([3.1303, -0.5664, 2.8519, 2.4305, -0.5627, -0.3391])

In [0]:
regressor.intercept

Out[19]: 2.7772157848801884

In [0]:
## Predictions
pred_results=regressor.evaluate(test_data)

In [0]:
## Final COmparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,2],[1.47,2.0])|     10.77|13.082579409753464|
| (6,[0,2],[2.0,2.0])|     12.69|  14.7416205326507|
|(6,[0,2],[3.15,3.0])|     20.08| 21.19336284697857|
|(6,[0,2],[3.18,2.0])|     19.82|18.435334730799262|
|(6,[0,2],[6.73,4.0])|     48.27| 35.25165219647679|
|[1.0,1.0,1.0,1.0,...|      3.07|10.623440910313029|
|[1.01,1.0,2.0,0.0...|     16.99|10.513529816573342|
|[1.48,0.0,2.0,0.0...|      8.52|11.649350994478496|
|[1.5,1.0,2.0,0.0,...|     10.65|11.145523856248143|
|[1.57,0.0,2.0,0.0...|     15.42|12.832911391181803|
|[1.63,1.0,2.0,0.0...|     11.87|11.552458471298408|
|[1.8,1.0,2.0,0.0,...|     12.43| 12.08460373713337|
|[1.92,0.0,1.0,1.0...|      8.58| 12.04249261847922|
|[2.0,0.0,2.0,0.0,...|     13.03|13.277089454679558|
|[2.0,0.0,3.0,1.0,...|     30.06|20.024011852088815|
|[2.0,1.0,2.0,0.0,...|     11.38|12.7106569910

In [0]:
### Performance matrix
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

Out[23]: (0.642605551272214, 4.515819736085914, 36.32891814773457)